In [1]:
import numpy as np
import cv2
import pickle
from datetime import datetime

In [2]:
#load cascade classifier training file for haarcascade
xml_path = 'C:/Anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(xml_path)

In [3]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("./face-trainner.yml")

In [4]:
labels = {}
with open("face-labels.pickle", 'rb') as f:
    og_labels = pickle.load(f)
    labels = {v:k for k,v in og_labels.items()}

labels

{0: 'chandler_bing',
 1: 'joey_tribbiani',
 2: 'monica_geller',
 3: 'munendra',
 4: 'phoebe_buffay',
 5: 'rachel_green',
 6: 'ross_geller'}

In [5]:
# Function to detect and recognize faces in an image

def detect_face_img(img):
    
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)

    for (x,y,w,h) in faces:
        #Draw rectangle around faces
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

        #Try to recognize face
        roi_gray = gray[y:y+h, x:x+w] 
        id_, conf = recognizer.predict(roi_gray)
        #print(conf, id_)

        #if conf>=50 and conf<=95:
        font = cv2.FONT_HERSHEY_SIMPLEX 
        name = str(labels[id_])
        cv2.putText(img, name, (x,y), font, 1, (255, 255, 255), 2)        
        
    return img

In [6]:
# Function to detect and recognize faces in a frame

def detect_face_vid(frame):
    
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
        
    if len(faces):
        if ts[2] == False:
            ts[0] = datetime.now()
            ts[2] = True

        for (x,y,w,h) in faces:
            #Draw rectangle around faces
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            #Try to recognize face
            roi_gray = gray[y:y+h, x:x+w] 
            id_, conf = recognizer.predict(roi_gray)
            #print(conf, id_)

            #if conf>=50 and conf<=95:
            font = cv2.FONT_HERSHEY_SIMPLEX 
            name = str(labels[id_])
            cv2.putText(frame, name, (x,y), font, 1, (255, 255, 255), 2) 
            ts[3] = name
    else:
        if ts[2] == True:
            ts[1] = datetime.now()
            ts[2] = False
            diff = ts[1]-ts[0]
            faceFound.append((ts[3], diff.seconds))
            #faceFound[labels[id_]] = diff.seconds
            print('Face Detected. From:',ts[0].strftime("%c"),' to:',ts[1].strftime("%c"))
        pass        
        
    return frame

### For Images

In [ ]:
img = cv2.imread('C:/Users/Dell/Desktop/FlipRobo/FRP02-FaceRecognition/test.jpg')
detected_img = detect_face_img(img)
# Display the resulting image with detected faces
cv2.imshow('Result',detected_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### For Video

In [8]:
# Capture time duration of faces detected
faceFound = []
ts = [0,0,False,'Unknown']

# video_capture = cv2.VideoCapture("C:/Users/Administrator/Desktop/FlipRobo/FRP02-FaceRecognition/friends.mp4")
video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()
    if ret:
        frame_with_face=detect_face_vid(frame)
        cv2.imshow("Result",frame_with_face)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
video_capture.release()
cv2.destroyAllWindows()

faceFound

Face Detected. From: Sat Jan 11 23:09:32 2020  to: Sat Jan 11 23:09:44 2020
Face Detected. From: Sat Jan 11 23:09:45 2020  to: Sat Jan 11 23:09:53 2020
Face Detected. From: Sat Jan 11 23:09:55 2020  to: Sat Jan 11 23:10:01 2020


[('chandler_bing', 11), ('munendra', 8), ('chandler_bing', 5)]